# Train the model

> This notebook is a simplified version of our workflow. It exposes the basic details of the traning and evaluation loop more explicitly, but does not offer advanced features like early stopping, mini-batches or validation. Use the `*-lightning` version for those.

First, we have to create the PyTorch objects out of the NPZ files. NPZ files behave like dictionaries of arrays. In our case, they contain two keys:

- `X`: the featurized systems
- `y`: the associated measurements

We can pass those dict-like arrays to an adapter class for Torch Datasets, which will be ingested by the DataLoaders. We also need the corresponding observation models.

## Define hyper parameters

Edit `UPPERCASE` variables in the following cells to configure behavior of the training.

In [1]:
DATASET = "ChEMBL"
WITH_OBSERVATION_MODEL = True
# Adam
LEARNING_RATE = 0.001
EPSILON = 1e-7
BETAS = 0.9, 0.999
# Trainer
MAX_EPOCHS = 50
N_SPLITS = 5
SHUFFLE_FOLDS = False
VALIDATION = False  # TODO: VALIDATION=True is not implemented yet!
MIN_ITEMS_PER_DATASET = 50  # skip datasets if len(data) < N
# Bootstrapping
N_BOOTSTRAPS = 1
BOOTSTRAP_SAMPLE_RATIO = 1
# Output
VERBOSE = False

In [2]:
from importlib import import_module
# Model -- specified with the full import path to the calss object
MODEL_CLS = "kinoml.ml.torch_models.ConvolutionNeuralNetworkRegression"
MODEL_KWARGS = {"hidden_size": 350}  # input_size is defined dynamically during trainin

⚠ From here on, you should _not_ need to modify anything else 🤞

In [3]:
model_module, model_class = MODEL_CLS.rsplit(".", 1)
ModelCls =  getattr(import_module(model_module), model_class)

In [4]:
# TODO: This should be specified along the tensor files as metadata, and should not depend on the dataset identity
MEASUREMENT_TYPES = {
    "ChEMBL": ["pKiMeasurement", "pIC50Measurement", "pKdMeasurement"],
    "PKIS2": ["PercentageDisplacementMeasurement"]
}[DATASET]

# TODO: Make all datasets use the same kinase identifiers
ONE_KINASE = {
    "ChEMBL": "P35968",
    "PKIS2": "ABL2",
}[DATASET]

In [5]:
# Nasty trick: save all-caps local variables (CONSTANTS working as hyperparametrs) so far in a dict to save it later
_hparams = {key: value for key, value in locals().items() if key.upper() == key and not key.startswith(("_", "OE_"))}

In [6]:
from pathlib import Path
from collections import defaultdict
import numpy as np
import shutil
import time

from IPython.display import Markdown
import pandas as pd
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
import pytorch_lightning as pl

from kinoml.utils import seed_everything
from kinoml.core import measurements as measurement_types
from kinoml.datasets.torch_datasets import XyNpzTorchDataset
from kinoml.core.measurements import null_observation_model

HERE = Path(_dh[-1])
_trial = 0
OUT = HERE / "_output" / DATASET / f"{time.time():.0f}"
OUT.mkdir(parents=True, exist_ok=True)
print("Reporting results at path:", OUT)
# Fix the seed for reproducible random splits -- otherwise we get mixed train/test groups every time, biasing the model evaluation
seed_everything()

Reporting results at path: /Users/taliakimber/Documents/github/experiments-binding-affinity/001_ligand-based/MorganFingerprint/LNN/_output/ChEMBL/1607069054


## Load featurized data and create observation models

In [9]:
datasets = defaultdict(dict)
for npz in HERE.glob(f"../_output/{DATASET}__*.npz"):
    _, kinase, measurement_type = str(npz.stem).split("__")
    datasets[kinase][measurement_type] = ds = XyNpzTorchDataset(npz)
#    if not VALIDATION:
#        ds.indices["test"] = np.concatenate([ds.indices["test"], ds.indices["val"]])
#        ds.indices["val"] = np.array([])

In [10]:
backend = "pytorch" if WITH_OBSERVATION_MODEL else "null"
obs_models = {k: getattr(measurement_types, k).observation_model(backend=backend) for k in MEASUREMENT_TYPES}
obs_models

{'pKiMeasurement': <function kinoml.core.measurements.pKiMeasurement._observation_model_pytorch(dG_over_KT, standard_conc=1, **kwargs)>,
 'pIC50Measurement': <function kinoml.core.measurements.pIC50Measurement._observation_model_pytorch(dG_over_KT, substrate_conc=1e-06, michaelis_constant=1, standard_conc=1, **kwargs)>,
 'pKdMeasurement': <function kinoml.core.measurements.pKdMeasurement._observation_model_pytorch(dG_over_KT, standard_conc=1, **kwargs)>}

## Check X duplication

There's a chance we have several measurements per ligand, or, depending on the featurization scheme, even hash collisions... Let's quantify the amount of input tensor duplication we are facing.

In [11]:
for mtype in MEASUREMENT_TYPES:
    display(Markdown(f"#### {mtype}"))
    unique = {}
    for kinase, dataset_by_mtype in datasets.items():
        if mtype in dataset_by_mtype:
            ds = dataset_by_mtype[mtype]
            all_ = ds.data_X.shape[0]
            unique_ = np.unique(ds.data_X, axis=0).shape[0]
            unique[kinase] = {"all": all_, "unique": unique_}
    df = pd.DataFrame.from_dict(unique).T
    df["uniqueness"] = df["unique"] / df["all"]
    # This is how you highlight rows in pandas!
    df = df.describe().style.apply(lambda x: ['font-weight: bold' for v in x], subset=pd.IndexSlice[["mean", "std"], :])
    display(df)

#### pKiMeasurement

,all,unique,uniqueness
count,177.000000,177.000000,177.000000
mean,91.576271,73.011299,0.908140
std,213.260181,159.515352,0.145384
min,1.000000,1.000000,0.333333
25%,2.000000,2.000000,0.873303
50%,11.000000,8.000000,1.000000
75%,69.000000,64.000000,1.000000
max,1497.000000,1125.000000,1.000000


#### pIC50Measurement

,all,unique,uniqueness
count,347.000000,347.000000,347.000000
mean,490.256484,387.616715,0.880322
std,1062.253429,814.083477,0.126062
min,1.000000,1.000000,0.462025
25%,6.000000,6.000000,0.815516
50%,68.000000,59.000000,0.893617
75%,399.000000,321.000000,1.000000
max,9626.000000,7252.000000,1.000000


#### pKdMeasurement

,all,unique,uniqueness
count,385.000000,385.000000,385.000000
mean,44.285714,26.490909,0.726013
std,70.687814,20.648107,0.155696
min,1.000000,1.000000,0.084726
25%,19.000000,15.000000,0.635897
50%,31.000000,22.000000,0.707692
75%,47.000000,33.000000,0.833333
max,838.000000,155.000000,1.000000


Now that we have all the data-dependent objects, we can start with the model-specific definitions.

### Training loop

In [12]:
from kinoml.ml.lightning_modules import KFold3Way, KFold
from IPython.display import Markdown
from tqdm.auto import trange, tqdm
from kinoml.ml.torch_models import NeuralNetworkRegression
from ipywidgets import HBox, VBox, Output, HTML
from kinoml.analysis.plots import predicted_vs_observed, performance
from kinoml.utils import fill_until_next_multiple
import pandas as pd
import torch.nn as nn

if VALIDATION:
    kfold = KFold3Way(n_splits=N_SPLITS, shuffle=SHUFFLE_FOLDS)
    ttypes = ["train", "val", "test"]
else:
    kfold = KFold(n_splits=N_SPLITS, shuffle=SHUFFLE_FOLDS)
    ttypes = ["train", "test"]

kinase_metrics = defaultdict(dict)
for kinase in tqdm(datasets):
    for mtype in MEASUREMENT_TYPES:
        if mtype not in datasets[kinase]:
            continue
        if datasets[kinase][mtype].data_X.shape[0] < MIN_ITEMS_PER_DATASET:
            print("Ignoring", kinase, "because it has less than", MIN_ITEMS_PER_DATASET, "entries for type", mtype)
            continue
            
        if VERBOSE:
            display(Markdown(f"#### {mtype}"))
        dataset = datasets[kinase][mtype]
        obs_model = obs_models[mtype]
        mtype_class = getattr(measurement_types, mtype)
        metrics = defaultdict(list)

        for fold_index, splits in enumerate(kfold.split(dataset.data_X, dataset.data_y)):
            if VALIDATION:
                train_indices, val_indices, test_indices = splits
            else:
                train_indices, test_indices = splits
            
            if VERBOSE:
                display(Markdown(f"##### Fold {fold_index}"))

            ####
            # TRAIN
            ####
            x_train = dataset.data_X[train_indices].float()
            x_test = dataset.data_X[test_indices].float()
            y_train = dataset.data_y[train_indices]
            y_test = dataset.data_y[test_indices]
            
            if VALIDATION:
                x_val = dataset.data_X[val_indices].float()
                y_val = dataset.data_y[val_indices]
                
            nn_model = ModelCls(input_size=x_train.shape[1], **MODEL_KWARGS)
            nn_model.train(True)

            optimizer = torch.optim.Adam(nn_model.parameters(), lr=LEARNING_RATE, eps=EPSILON, betas=BETAS)
            loss_function = torch.nn.MSELoss()
            
            if VERBOSE:
                range_epochs = trange(MAX_EPOCHS, desc="Epochs (+ featurization...)")
            else:
                range_epochs = range(MAX_EPOCHS)
            for epoch in range_epochs:
                optimizer.zero_grad()

                prediction = nn_model(x_train)
                if WITH_OBSERVATION_MODEL:
                    prediction = obs_model(prediction)

                prediction = prediction.view_as(y_train)

                # prediction = delta_g
                loss = loss_function(prediction, y_train)
                if VERBOSE:
                    range_epochs.set_description(f"Epochs (loss={loss.item():.2e})")
                
                if VALIDATION:
                    raise NotImplementedError("Validation step not implemented yet")
                    
                
                # Gradients w.r.t. parameters
                loss.backward()

                # Optimizer
                optimizer.step()

            ####
            # EVAL
            ####
            nn_model.eval()
            outputs = []
            for ttype in ttypes:
                output = Output()
                with output:
                    title = f"fold={fold_index}, {ttype}={locals()[f'{ttype}_indices'].shape[0]}"
                    print(title)
                    print("-"*(len(title)))

                    observed = locals()[f"y_{ttype}"]

                    with torch.no_grad():
                        predicted = nn_model(locals()[f"x_{ttype}"])
                        if WITH_OBSERVATION_MODEL:
                            predicted = obs_model(predicted)

                    predicted = predicted.view_as(observed).detach().numpy()
                    observed = observed.detach().numpy()
                    these_metrics = performance(predicted, observed, n_boot=N_BOOTSTRAPS, sample_ratio=BOOTSTRAP_SAMPLE_RATIO)
                    metrics[ttype].append(these_metrics)
                    if VERBOSE:
                        display(predicted_vs_observed(predicted, observed, mtype_class, with_metrics=False))

                outputs.append(output)
            if VERBOSE:
                display(HBox(outputs))

        # Average performances
        
        average = defaultdict(dict)
        for key in metrics["test"][0]:
            for label in ttypes:
                # this zero here ---v is super important! we only want the mean of the means!
                values =  [fold[key][0] for fold in metrics[label]]
                average[label][key] = {
                    "mean": np.mean(values),
                    "std": np.std(values)
                }
        if VERBOSE:
            for label in ttypes:    
                display(HTML(f"Bootstrapped average across folds ({label}):"))
                display(pd.DataFrame.from_dict(average[label]))
        kinase_metrics[kinase][mtype] = average

Ignoring P20794 because it has less than 50 entries for type pKdMeasurement
Ignoring P31751 because it has less than 50 entries for type pKiMeasurement
Ignoring P31751 because it has less than 50 entries for type pKdMeasurement
Ignoring Q58F21 because it has less than 50 entries for type pKiMeasurement
Ignoring Q58F21 because it has less than 50 entries for type pKdMeasurement
Ignoring Q9NR20 because it has less than 50 entries for type pIC50Measurement
Ignoring Q04912 because it has less than 50 entries for type pKiMeasurement
Ignoring Q04912 because it has less than 50 entries for type pKdMeasurement
Ignoring Q12866 because it has less than 50 entries for type pKiMeasurement
Ignoring P27448 because it has less than 50 entries for type pKiMeasurement
Ignoring P27448 because it has less than 50 entries for type pIC50Measurement
Ignoring P27448 because it has less than 50 entries for type pKdMeasurement
Ignoring Q8IZX4 because it has less than 50 entries for type pKiMeasurement
Ignoring

### Summary

`kinase_metrics` is a nested dictionary with these dimensions:

- kinase name
- measurement type
- metric
- mean & standard deviation

In [13]:
display(Markdown(f"### {DATASET}, observation model = {WITH_OBSERVATION_MODEL}"))
for mtype in MEASUREMENT_TYPES:
    display(Markdown(f"#### {mtype}"))
    # This is going to be fun:
    df = pd.concat({kinase_name: 
                    pd.DataFrame.from_dict(
                        {f"{train_test}_{metric}_{stat}": (value,) 
                         for train_test, vv in v[mtype].items() 
                         for metric, vvv    in vv.items() 
                         for stat, value    in vvv.items()}
                    ).assign(zeros=(datasets[kinase_name][mtype].data_y == 0).sum().detach().numpy())        
                    for kinase_name, v in sorted(kinase_metrics.items(), key=lambda kv: kv[0].lower())
                    if mtype in v})
    
    df.index = [index[0] for index in df.index]
    with pd.option_context("display.float_format", "{:.3f}".format, "display.max_rows", len(df)):
        display(df.style.background_gradient(subset=["train_r2_mean", "test_r2_mean"], low=0, high=1, vmin=0, vmax=1))

### ChEMBL, observation model = True

#### pKiMeasurement

,train_mae_mean,train_mae_std,train_mse_mean,train_mse_std,train_r2_mean,train_r2_std,train_rmse_mean,train_rmse_std,test_mae_mean,test_mae_std,test_mse_mean,test_mse_std,test_r2_mean,test_r2_std,test_rmse_mean,test_rmse_std,zeros
O00311,1.113843,0.044482,1.706735,0.182976,-0.563899,0.171676,1.304480,0.071188,1.809561,1.122077,5.402326,6.520280,-8.442070,10.730749,2.013481,1.161128,0
O14757,0.990512,0.048064,1.536169,0.180460,0.106201,0.143725,1.237422,0.070400,1.108024,0.198957,1.915851,0.639392,-0.649673,1.411285,1.364281,0.233639,0
O14965,0.805810,0.030138,1.000873,0.101964,0.388638,0.090350,0.999156,0.050607,1.442392,0.555359,2.999394,1.840221,-1.971542,2.559779,1.645125,0.541257,0
O60674-O60674,0.857279,0.096952,1.177877,0.242510,0.190226,0.145069,1.078358,0.122562,1.042575,0.294239,1.709917,0.920439,-3.055633,2.982689,1.257949,0.357045,0
O60885,0.780740,0.037631,0.934133,0.142977,0.139303,0.122581,0.963875,0.071255,1.005084,0.409663,1.608890,1.268199,-1.290077,2.326071,1.194845,0.425718,0
O75116,1.013948,0.125046,1.644691,0.320858,-0.362589,0.270593,1.275271,0.135553,1.243987,0.493388,2.413531,2.037266,-3.389940,2.557083,1.441816,0.578530,0
O96013,0.589139,0.017286,0.564716,0.046758,0.282329,0.133065,0.750791,0.032076,0.981107,0.376123,1.665055,1.196581,-1.223621,0.994952,1.209825,0.448753,0
P00519,1.077890,0.056105,1.807892,0.167445,0.033813,0.175319,1.343141,0.062170,1.288593,0.394861,2.532009,1.453324,-2.096653,1.753138,1.526231,0.450141,0
P00533,0.642244,0.055924,0.714590,0.117985,0.666902,0.101582,0.842495,0.069233,0.858354,0.180580,1.286239,0.616975,0.150886,0.338200,1.102518,0.265884,0
P05129,0.481463,0.034946,0.423081,0.060405,0.707559,0.087661,0.648713,0.047463,1.054204,0.761662,2.037180,2.542009,-0.679409,1.661396,1.222212,0.737142,0


#### pIC50Measurement

,train_mae_mean,train_mae_std,train_mse_mean,train_mse_std,train_r2_mean,train_r2_std,train_rmse_mean,train_rmse_std,test_mae_mean,test_mae_std,test_mse_mean,test_mse_std,test_r2_mean,test_r2_std,test_rmse_mean,test_rmse_std,zeros
O00141,0.871667,0.068807,1.164488,0.214805,0.129685,0.106317,1.074689,0.097631,1.161670,0.502895,2.058771,1.486395,-1.505506,2.213200,1.337974,0.518263,0
O00311,0.974854,0.053217,1.425072,0.114427,-0.498201,0.221294,1.192750,0.049172,0.988942,0.077306,1.537180,0.186726,-1.344807,1.239877,1.237390,0.077764,0
O00418,0.459621,0.096918,0.389416,0.119291,0.452447,0.149544,0.616194,0.098597,0.589052,0.379068,1.194302,1.860377,-1.757532,2.059720,0.842368,0.696217,0
O00444,0.970566,0.130551,1.463825,0.355108,0.399228,0.045167,1.199817,0.155767,1.025059,0.427827,2.040218,2.032255,-0.324784,0.583890,1.292870,0.607210,0
O14757,0.941658,0.009940,1.451889,0.034119,0.075604,0.042612,1.204860,0.014149,1.031404,0.080960,1.759742,0.343979,-0.276913,0.336703,1.320485,0.126731,0
O14920,0.838131,0.010635,1.125515,0.053997,0.152645,0.038511,1.060598,0.025446,0.895460,0.057865,1.255597,0.122376,-0.037962,0.235609,1.119211,0.054443,0
O14965,1.050906,0.013506,1.739883,0.032581,0.036910,0.097834,1.318989,0.012337,1.220845,0.092027,2.419549,0.321716,-0.681197,0.479597,1.552067,0.103138,0
O15111,0.712567,0.050230,0.824952,0.118967,-0.229832,0.232801,0.905820,0.066650,0.878505,0.169284,1.279058,0.528347,-1.798103,1.699913,1.108435,0.224565,0
O15164,0.388538,0.036328,0.225238,0.046538,0.746451,0.027452,0.472131,0.048275,0.561409,0.186738,0.469906,0.286353,-0.877997,2.063156,0.653404,0.207292,0
O15264,0.538374,0.042043,0.416247,0.066465,0.674305,0.035060,0.643184,0.050608,0.808907,0.172107,1.106960,0.439815,-0.921483,1.308107,1.031028,0.209623,0


#### pKdMeasurement

,train_mae_mean,train_mae_std,train_mse_mean,train_mse_std,train_r2_mean,train_r2_std,train_rmse_mean,train_rmse_std,test_mae_mean,test_mae_std,test_mse_mean,test_mse_std,test_r2_mean,test_r2_std,test_rmse_mean,test_rmse_std,zeros
O00444,0.679206,0.099093,0.758448,0.156156,0.386449,0.060265,0.866555,0.086776,1.199328,0.236055,2.142552,0.722844,-0.686044,0.844167,1.436046,0.283413,0
O14578,0.479008,0.050536,0.409163,0.097946,0.243283,0.108492,0.634557,0.080627,0.706729,0.245148,0.878261,0.668243,-0.895428,1.444410,0.872891,0.341061,0
O14965,0.737605,0.059077,0.955621,0.135004,0.263182,0.105982,0.975012,0.070509,1.027294,0.619928,2.474960,2.361026,-3.928862,6.202038,1.401833,0.714020,0
O14976,0.752422,0.030853,0.895783,0.078846,-0.271431,0.131351,0.945526,0.041991,0.927062,0.279700,1.387593,0.583981,-0.871015,0.634622,1.154034,0.236219,0
O15530,0.572842,0.033004,0.558899,0.028845,0.690165,0.050958,0.747347,0.019271,0.930041,0.328070,1.319889,0.844718,-0.396457,0.896102,1.093140,0.353460,0
O43293,0.735469,0.069859,0.866589,0.122202,0.268670,0.067705,0.928616,0.065278,1.131606,0.336856,2.256485,1.366727,-1.713097,1.891767,1.439783,0.428381,0
O43318,0.698717,0.049363,0.697195,0.081112,0.349273,0.100866,0.833580,0.048361,1.337562,0.448224,2.871916,1.972855,-2.275741,1.993700,1.604815,0.544505,0
O43353,0.460997,0.063564,0.396391,0.099530,0.300904,0.132561,0.625078,0.075289,0.627561,0.299426,0.830413,0.634036,-0.429491,1.021458,0.824253,0.388613,0
O60674-O60674,0.868190,0.098124,1.205810,0.268690,0.254692,0.139755,1.091431,0.120783,1.237265,0.407896,2.164416,1.235037,-0.716797,0.736168,1.416943,0.395840,0
O60885,0.605278,0.030106,0.730526,0.108144,0.301154,0.163846,0.852091,0.066838,0.921438,0.280083,1.370375,0.657869,-0.547544,0.849021,1.134294,0.289398,0


#### Overall performance

In [14]:
df[["train_r2_mean", "train_r2_std", "test_r2_mean", "test_r2_std", "zeros"]].describe().style.apply(lambda x: ['font-weight: bold' for v in x], subset=pd.IndexSlice[["mean", "std"], :])

,train_r2_mean,train_r2_std,test_r2_mean,test_r2_std,zeros
count,86.000000,86.000000,86.000000,86.000000,86.000000
mean,0.298668,0.119168,-1.672202,1.715505,0.000000
std,0.253591,0.081190,2.120728,2.363841,0.000000
min,-0.472747,0.022100,-17.700970,0.206432,0.000000
25%,0.149579,0.068537,-2.172167,0.756858,0.000000
50%,0.303051,0.105494,-1.156520,1.092108,0.000000
75%,0.437138,0.145630,-0.614787,1.837181,0.000000
max,0.861999,0.537738,0.322888,19.453969,0.000000


### Save reports to disk

In [15]:
%%capture cap --no-stderr
from kinoml.utils import watermark
import json

df.to_csv(OUT / "performance.csv")

with open(OUT / "performance.json", "w") as f:
    json.dump(kinase_metrics, f)

In [16]:
w = watermark()
with open(OUT/ "watermark.txt", "w") as f:
    f.write(cap.stdout)

with open(OUT / "hparams.json", "w") as f:
    json.dump(_hparams, f, default=str)

Watermark
---------
Last updated: 2020-12-04T10:00:31.803062+01:00

Python implementation: CPython
Python version       : 3.7.8
IPython version      : 7.19.0

Compiler    : Clang 11.0.0 
OS          : Darwin
Release     : 19.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Hostname: Talias-MacBook-Pro.local

Git hash: ab848d47088e039875c021ea989b18861b5f2948

kinoml           : 0+unknown
torch            : 1.6.0
pandas           : 1.1.4
json             : 2.0.9
pytorch_lightning: 1.0.8
numpy            : 1.19.4

Watermark: 2.1.0


conda
-----
sys.version: 3.7.7 (default, Mar 23 2020, 17:31:31) 
...
sys.prefix: /opt/miniconda3
sys.executable: /opt/miniconda3/bin/python
conda location: /opt/miniconda3/lib/python3.7/site-packages/conda
conda-build: None
conda-env: /opt/miniconda3/bin/conda-env
user site dirs: 

CIO_TEST: <not set>
CONDA_DEFAULT_ENV: kinoml-pinned-by-commit
CONDA_EXE: /opt/miniconda3/bin/conda
CONDA_MKL_INTERFACE_LAYER_BACKUP: 
CONDA_PREFIX: